In [ ]:
try:
    # %tensorflow_version only exists in Colab.
    !pip install tf-nightly
except Exception:
    pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


In [ ]:
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"


In [ ]:
def preprocess_data(file_path):
    data = pd.read_csv(file_path, sep="\t", header=None, names=["label", "message"])
    data['label'] = data['label'].map({'ham': 0, 'spam': 1})
    return data

# Load and preprocess the data
train_data = preprocess_data(train_file_path)
test_data = preprocess_data(test_file_path)

# Tokenization and Vectorization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 200
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['message'])
train_sequences = tokenizer.texts_to_sequences(train_data['message'])
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_len)

test_sequences = tokenizer.texts_to_sequences(test_data['message'])
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_len)

# Build the model
model = keras.Sequential([
    keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=max_len),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(train_padded, train_data['label'], epochs=5, validation_data=(test_padded, test_data['label']))

# Function to predict message and return likelihood and label
def predict_message(pred_text):
    pred_seq = tokenizer.texts_to_sequences([pred_text])
    pred_padded = pad_sequences(pred_seq, padding='post', maxlen=max_len)
    prediction_prob = model.predict(pred_padded)

    # Return probability and label
    if prediction_prob[0] < 0.5:
        return [float(1 - prediction_prob[0]), 'ham']
    else:
        return [float(prediction_prob[0]), 'spam']

# Example prediction
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
def test_predictions():
    test_messages = ["how are you doing today",
                     "sale today! to stop texts call 98912460324",
                     "i dont want to go. can we try it a different day? available sat",
                     "our new mobile video service is live. just install on your phone to start watching.",
                     "you have won £1000 cash! call to claim your prize.",
                     "i'll bring it tomorrow. don't forget the milk.",
                     "wow, is your arm alright. that happened to me one time too"
                    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

test_predictions()
